In [11]:
'-p', 'C:/Users/Elitebook/Downloads/my_data'


('-p', 'C:/Users/Elitebook/Downloads/my_data')

In [16]:
import pandas as pd
import os

# Path to your downloaded Dakshina Hindi dataset
input_folder = r"C:\Users\Elitebook\nashra Dropbox\Nashra\PC\Downloads\dakshina_dataset_v1.0\dakshina_dataset_v1.0\hi\lexicons"
output_folder = r"C:\Users\Elitebook\Downloads\my_data"

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Map from original .tsv file names to desired .csv output file names
file_map = {
    "hi.translit.sampled.train.tsv": "hi_train.csv",
    "hi.translit.sampled.dev.tsv": "hi_valid.csv",
    "hi.translit.sampled.test.tsv": "hi_test.csv"
}

# Convert each file
for tsv_file, csv_file in file_map.items():
    tsv_path = os.path.join(input_folder, tsv_file)
    csv_path = os.path.join(output_folder, csv_file)

    # Read TSV and save as CSV
    df = pd.read_csv(tsv_path, sep='\t', header=None)
    df.to_csv(csv_path, index=False, header=False)

    print(f"Converted {tsv_file} to {csv_file}")


Converted hi.translit.sampled.train.tsv to hi_train.csv
Converted hi.translit.sampled.dev.tsv to hi_valid.csv
Converted hi.translit.sampled.test.tsv to hi_test.csv


In [2]:
!pip install pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.1/823.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 41.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

In [22]:
import wandb
wandb.login(key="c32cc9e7b9c39295b3b1821cf5bf4fe508d41045")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Elitebook\_netrc
wandb: Currently logged in as: amaannashra012 (amaannashra012-iit-madras-alumni-association) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [23]:
import numpy as np
import pandas as pd
import torch
import random
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pytorch_lightning as L
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.loggers import WandbLogger
import csv
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import os
import wandb
import argparse
from Accesories_functions import trainDataset,Helper_functions
from Encoder import encoder
from Vanila_decoder import decoder
from Encoder_Decoder_vanilla_seq import eng_ben
from Attention_Decoder import decoder_with_Attention
from seq_to_seq_attention import eng_ben_attention
wandb.login()


if torch.cuda.is_available():
    device = torch.device('cuda')  # Get the GPU device
else:
    device = torch.device('cpu')   # Use CPU if GPU is not available

def main(args):
    if 145==145:
        obj=Helper_functions()
        #config=wandb.config
        #wandb.run.name='bs-'+str(config.batch_size)+'-ct-'+ str(config.cell_type)+'-ep-'+str(config.epochs)+ '-es-'+str(config.embedding_size)+'-hs-'+str(config.hidden_size)+'-nel-'+str(config.encoder_num_layers)+'-ndl-'+str(config.decoder_num_layers)+'-do-'+str(config.drop_out)+'-bd-'+str(config.bidirectional)
        path=args.path
        # Extracting and preprocessing training, validation, and test data
        train_data = obj.Extract_data(os.path.join(path, 'hi_train.csv'))
        train_test_data = obj.Extract_data(os.path.join(path, 'hi_test.csv'))
        train_val_data = obj.Extract_data(os.path.join(path, 'hi_valid.csv'))
        encoder_input_text,decoder_input_text,encoder_char_to_index,decoder_char_to_index,encoder_vocab,decoder_vocab=obj.helper(train_data)
        # Creating character-to-index and index-to-character mappings for encoder and decoder
        decoder_index_to_char={index: char for char, index in decoder_char_to_index.items()}
        encoder_index_to_char={index: char for char, index in encoder_char_to_index.items()}
        batch_size=args.batch_size
        # Creating training, validation, and test datasets and dataloaders
        p,p1=obj.words_to_tensor(encoder_input_text,encoder_char_to_index,decoder_input_text,decoder_char_to_index)
        dataset1=trainDataset(p,p1)
        dataloader=DataLoader(dataset=dataset1,batch_size=batch_size,shuffle=True,num_workers=1)
        encoder_input_text_test,decoder_input_text_test,_,_,_,_=obj.helper(train_test_data)
        encoder_input_text_val,decoder_input_text_val,_,_,_,_=obj.helper(train_val_data)
        p_test,p1_test=obj.words_to_tensor(encoder_input_text_test,encoder_char_to_index,decoder_input_text_test,decoder_char_to_index)
        testdataset=trainDataset(p_test,p1_test)  #create train dataset
        testDataloader=DataLoader(dataset=testdataset,batch_size=batch_size,shuffle=False,num_workers=1)
        p_val,p1_val=obj.words_to_tensor(encoder_input_text_val,encoder_char_to_index,decoder_input_text_val,decoder_char_to_index)
        val_dataset=trainDataset(p_val,p1_val)
        valDataloader=DataLoader(dataset=val_dataset,batch_size=batch_size,shuffle=False,num_workers=1)
        heatMap_dataloader=DataLoader(dataset=testdataset,batch_size=9,shuffle=True,num_workers=1)
        first_batch = next(iter(heatMap_dataloader))
         # Retrieving model parameters from command-line arguments
        cell_type=args.cell_type
        embedding_size=args.embedding_size
        hidden_size=args.hidden_size
        encoder_num_layer=args.encoder_num_layer
        decoder_num_layer=args.decoder_num_layer
        drop_out=args.dropout_rate
        epoch=args.epochs
        bidirectional=args.bidirectional
        max_seq_length=p.shape[1]
        # Initializing Wandb
        wandb.init(entity=args.wandb_entity,project=args.project_name)
        wandb_logger = WandbLogger(project=args.project_name, entity=args.wandb_entity)
        # Initializing encoder and decoder models
        en=encoder(cell_type,encoder_vocab,embedding_size,hidden_size,encoder_num_layer,drop_out,bidirectional)
        de=decoder(cell_type,decoder_vocab,embedding_size,hidden_size,decoder_num_layer,drop_out,epoch,0.5,bidirectional)
        de_att=decoder_with_Attention(max_seq_length,cell_type,decoder_vocab,embedding_size,hidden_size,decoder_num_layer,drop_out,epoch,0.5,bidirectional)
         # Selecting the model based on attention flag
        if args.attention==False:
            model=eng_ben(en,de,decoder_vocab,decoder_index_to_char,encoder_index_to_char)
        else:
            model=eng_ben_attention(en,de_att,decoder_vocab,decoder_index_to_char,encoder_index_to_char)

         # Training the model
        trainer = L.Trainer(accelerator='auto',devices="auto",max_epochs=epoch,num_sanity_val_steps=0,logger=wandb_logger)
        trainer.fit(model,dataloader,valDataloader)
        trainer.test(dataloaders=testDataloader,ckpt_path='best')  # Testing the model
        if args.attention==False and args.plot_heatmap==True:  # Printing a message if heatmap plotting is requested but attention is not applied
            print("attention has not applied to the model")
        elif args.attention==True and args.plot_heatmap==True:
            trainer.predict(dataloaders=heatMap_dataloader,ckpt_path='best')

if  __name__ =="__main__":
    parser = argparse.ArgumentParser()  #taking arguments from command line arguments
    # Adding command-line arguments
    parser.add_argument('-wp','--project_name',type=str,default='Assignment3_DL',help='Project name used to track experiments in Weights & Biases dashboard')
    parser.add_argument('-we','--wandb_entity',type=str,default='amaannashra012-iit-madras-alumni-association',help='Project name used to track experiments in Weights & Biases dashboard')
    parser.add_argument('-p','--path',type=str,help='provide the path where your data is stored in memory,Read the readme for more description')
    parser.add_argument('-e','--epochs',type=int,default=15,help='Number of epochs to train')
    parser.add_argument('-b','--batch_size',type=int,default=128,help='Batch size used to train RNN')
    parser.add_argument('-es','--embedding_size',type=int,default=256,help='default size of Enbedding Layer')
    parser.add_argument('-hs','--hidden_size',type=int,default=256,help='default size of context vector')
    parser.add_argument('-enl','--encoder_num_layer',type=int,default=2,help='number of encoder Layer')
    parser.add_argument('-dnl','--decoder_num_layer',type=int,default=1,help='number of decoder Layer')
    parser.add_argument('-ct','--cell_type',type=str,default='LSTM',choices=['GRU','RNN','LSTM'],help='which cell gonna use either GRU or LSTM or RNN')
    parser.add_argument('-do','--dropout_rate',type=float,default=0.3,choices=[0.2,0.3,0.4],help='drop out rate to regularilze the model')
    parser.add_argument('-bd','--bidirectional',type=bool,default=True,choices=[True,False],help='biderctional model or not')
    parser.add_argument('-a','--attention',type=int,default=1,choices=[1,0],help='We will use attention or not')
    parser.add_argument('-hp','--plot_heatmap',type=int,default=0,choices=[1,0],help='We will plot the heatmap or not')
    args = parser.parse_args(args=['-p', 'C:/Users/Elitebook/Downloads/my_data'])

    main(args)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\Elitebook\AppData\Local\Programs\Python\Python313\Lib\site-packages\pytorch_lightning\loggers\wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.

  | Name    | Type                   | Params | Mode 
-----------------------------------------------------------
0 | encoder | encoder                | 2.6 M  | train
1 | decoder | decoder_with_Attention | 2.1 M  | train
-----------------------------------------------------------
4.8 M     Trainable params
0         Non-trainable params
4.8 M     Total params
19.160    Total estimated model params size (MB)
13        Mo

Epoch 14: 100%|██████████| 346/346 [10:13<00:00,  0.56it/s, v_num=8h0n, val_loss=0.779, train_loss=0.355, val_acc=0.457]val accuracy is 0.4598439651216154


`Trainer.fit` stopped: `max_epochs=15` reached.


Epoch 14: 100%|██████████| 346/346 [10:13<00:00,  0.56it/s, v_num=8h0n, val_loss=0.779, train_loss=0.355, val_acc=0.457]


Restoring states from the checkpoint path at .\Assignment3_DL\tvat8h0n\checkpoints\epoch=14-step=5190.ckpt
Loaded model weights from the checkpoint at .\Assignment3_DL\tvat8h0n\checkpoints\epoch=14-step=5190.ckpt
c:\Users\Elitebook\AppData\Local\Programs\Python\Python313\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing DataLoader 0: 100%|██████████| 36/36 [00:14<00:00,  2.41it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.07877795398235321
        test_loss            0.773327648639679
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


## Question number 6 code

In [23]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_attention_matrix(input_seq, output_seq, attention_weights, title="Attention Heatmap"):
    """
    Plots an attention heatmap between input and output sequences.

    Args:
        input_seq (list): List of input characters (e.g., ['p', 'a', 'n', 'd', 'a']).
        output_seq (list): List of predicted output characters (e.g., ['ప', 'ా', 'ం']).
        attention_weights (Tensor or np.ndarray): Attention weights of shape (output_len, input_len).
        title (str): Title of the plot.
    """
    if isinstance(attention_weights, torch.Tensor):
        attention_weights = attention_weights.detach().cpu().numpy()
    
    
    plt.figure(figsize=(len(input_seq), len(output_seq)))
    sns.heatmap(attention_weights, xticklabels=input_seq, yticklabels=output_seq, cmap="viridis", annot=True, fmt=".2f")
    plt.xlabel("Input Sequence")
    plt.ylabel("Output Sequence")
    plt.title(title)
    plt.tight_layout()
    plt.show()
